
# HackerEarth Deep Learning Challenge: 'Tis STILL the season to be jolly

You work for a social media platform. Your task is to create a solution using deep learning to discern whether a post is holiday-related in an effort to better monetize the platform.

Task-
You are given the following six categories. You are required to classify the images in the dataset based on these categories.

    Miscellaneous
    Christmas_Tree
    Jacket
    Candle
    Airplane
    Snowman


# Loading necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.models import Model
import os
import cv2

In [ ]:
df_train = pd.read_csv('../input/hackereath-holiday-season-deep-learning-contest/dataset/train.csv')
df_train.head()

# Number of training classes

In [ ]:
df_train['Class'].value_counts()

# Training data augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
         shear_range=0.2,
         zoom_range=0.2,
         horizontal_flip=True,
         width_shift_range=0.2,
         height_shift_range=0.2)

train_generator=train_datagen.flow_from_dataframe(dataframe=df_train,
                                            directory="../input/hackereath-holiday-season-deep-learning-contest/dataset/train",
                                            x_col="Image",
                                            y_col="Class",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(299,299))

# Loading pre-trained models

In [ ]:
model = tf.keras.applications.InceptionResNetV2(include_top=True,weights="imagenet")

In [ ]:
x = model.layers[-2].output
fc = Dense(1024,activation='relu')(x)
fc = Dense(512,activation='relu')(fc)
fc = Dense(256,activation='relu')(fc)
fc = Dense(6,activation='softmax')(fc)

In [ ]:
model1 = Model(inputs=model.input,outputs=fc)
#model1.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model1, to_file='model.png', rankdir='LR')

In [ ]:
plot_model(model1, to_file='model.png')

In [ ]:
#freezing all the layers
for layer in model1.layers[:-16]:
    #print(l)
    layer.trainable = False

# Compiling model

In [ ]:
model1.compile(optimizer='adam',loss ="categorical_crossentropy",metrics=["accuracy"])

# Training model

In [ ]:
history = model1.fit_generator(train_generator, epochs=4)

In [ ]:
classes = train_generator.class_indices

In [ ]:
classes = dict((v,k) for k,v in classes.items())
classes = [classes[k] for k in classes]

In [ ]:
Image = []
prediction = []


for i, file in enumerate(os.listdir('../input/hackereath-holiday-season-deep-learning-contest/dataset/test/')):
    Image.append(file)
    file = '../input/hackereath-holiday-season-deep-learning-contest/dataset/test/' + file

    img = image.load_img(file, target_size=(299,299,3)) 
    img = image.img_to_array(img)
    img = img/255
    pred = model1.predict(img.reshape(1,299,299,3))

    prediction.append(classes[np.argmax(pred[0])])

# Submission file creation

In [ ]:
data=pd.DataFrame((zip(Image, prediction)),columns=['Image','Class'])
data.head()

In [ ]:
data.to_csv('submission.csv',index=False)